# 筆記本 1：從聊天記錄中提取和格式化事件資訊

### 目標
在此範例中，我們將從聊天記錄中提取與事件（如落石或停電）相關的資訊，並將其格式化為簡潔的摘要。

### 操作概要
1. **聊天記錄格式:** 我們的聊天記錄檔案，內容中記載了日期、事件類型及其描述。
2. 通過正則表達式來匹配聊天記錄中的日期和事件類型。
3. 將提取出的事件資料格式化為摘要並儲存至文件中。


In [ ]:
# 安裝必要的 Python 套件
!pip install pandas

In [ ]:
# 匯入必要的套件
import re
from collections import defaultdict
import pandas as pd

### 聊天記錄格式範例
假設聊天記錄的格式如下：

```plaintext
[LINE] Chat history in 華陵里里民（公務）平台
Saved on: 2024/04/05, 17:53

2023/05/29, Mon
09:19 AM		⁨⁨Keith Lee⁩⁩ invited ⁨⁨菁⁩⁩ to the group. Wait for them to join before chatting.
09:19 AM		⁨⁨菁⁩⁩ joined the group.
09:24 AM	陳豙殳	[Sticker]
11:34 AM	阿旭	"大家平安
里辦公處〔再次宣導〕
毛小孩結紮
謝謝大家🙂

有關〔家犬家貓結紮〕

請有需要的民眾，趕快找我我（里幹事-阿旭）登記喔，基於個資，記得〔請私賴我〕感謝您🙆‍♂️。

時間：
第一天：
```
我們將從這樣的格式中提取事件，並將它們分格式保存。

### 步驟 1：定義事件解析函數
以下函數會解析聊天記錄中的日期與事件描述，並將事件按照日期和類型儲存。


In [ ]:
def parse_events(chat_content):
    """
    解析聊天內容，提取特定事件的日期和描述。
    回傳以日期和事件類型為索引的事件字典。
    """
    events_by_date = defaultdict(lambda: defaultdict(list))
    current_date = None
    target_event_types = ['落石', '停電']  # 可根據需求擴充其他事件類型
    
    # 逐行處理聊天內容
    for line in chat_content.splitlines():
        # 檢查是否為日期行，例如：2024/11/08
        date_match = re.match(r'^(\d{4}/\d{2}/\d{2}),', line)
        if date_match:
            current_date = date_match.group(1)
        elif current_date:
            # 根據事件類型提取事件描述
            for event_type in target_event_types:
                if event_type in line:
                    event_details = re.sub(r'\d{2}:\d{2} (AM|PM)\t.*?\t', '', line).strip()
                    events_by_date[current_date][event_type].append(event_details)
    return events_by_date

# 測試解析函數
chat_content = '''
2023/06/05, Mon
05:31 AM		呂銘樹 unsent a message.
06:08 AM	芋頭	[Photo]
06:08 AM	芋頭	舅媽里長 下巴崚有落石
06:23 AM	華陵里里長 梁雅惠	[Sticker]
06:25 AM	華陵里里長 梁雅惠	已通報了
06:36 AM		吳世傑 unsent a message.
'''

parsed_events = parse_events(chat_content)
parsed_events

### 步驟 2：定義事件摘要格式化函數
此函數會將儲存的事件信息格式化，讓它們變成可讀的摘要，並儲存到文件中。


In [ ]:
def format_event_summary(events_by_date):
    """
    格式化事件字典，生成指定格式的事件摘要。
    """
    return '\n'.join(
        f"{date}，{event_type}，{'；'.join(details)}"
        for date, event_types in events_by_date.items()
        for event_type, details in event_types.items()
    )

# 測試格式化函數
formatted_event_summary = format_event_summary(parsed_events)
print(formatted_event_summary)

### 步驟 3：定義讀取和寫入檔案的輔助函數
我們需要編寫讀取檔案和將結果寫入檔案的輔助函數。

In [ ]:
def read_file(filepath):
    """讀取檔案內容並回傳字串。"""
    with open(filepath, 'r', encoding='utf-8') as file:
        return file.read()

def write_file(filepath, content):
    """將字串內容寫入指定的檔案。"""
    with open(filepath, 'w', encoding='utf-8') as file:
        file.write(content)

# 測試讀取與寫入文件
test_content = read_file('chat_history.txt')  # 請確認這檔案存在，內容可參照本文範例聊天記錄格式
print(test_content[:100])  # 印出前100字元查看

### 步驟 4：整合所有步驟至主函數
主函數將會將所有流程自動化：讀取文件 -> 解析 -> 格式化 -> 寫入。

In [ ]:
def main(input_file, output_file):
    # 讀取檔案
    chat_content = read_file(input_file)

    # 解析事件
    events_by_date = parse_events(chat_content)

    # 格式化事件摘要
    formatted_event_summary = format_event_summary(events_by_date)

    # 將事件摘要寫入輸出檔案
    write_file(output_file, formatted_event_summary)
    print(f'事件摘要已儲存到 {output_file}')

# 調用主函數進行完整程序操作
main('chat_history.txt', 'output_summary.txt')

### 結論
- 在本教程中，我們學習如何從聊天記錄中提取和格式化特定事件的資訊。
    - 格式化後的事件摘要將易於進行後續的分析工作（如事件趨勢、統計分析等）。
- 下一步可以將結果應用於更進階的可視化分析或數據處理。
